In [2]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import numpy as np
from io import BytesIO
from PIL import Image
import tensorflow as tf
import asyncio
import nest_asyncio


In [10]:
nest_asyncio.apply()
async def my_coroutine():
    app = FastAPI()

    origins = [
    "http://localhost/",
    "http://localhost:3000/",
]
    app.add_middleware(
        CORSMiddleware,
        allow_origins=origins,
        allow_credentials=True,
        allow_methods=[""],
        allow_headers=[""],
    )

    Model = tf.keras.models.load_model("../saved_models1/1")

    CLASS_NAMES = ["Tomato__Target_Spot", "Tomato__Tomato_mosaic_virus", "Tomato__Tomato_YellowLeaf__Curl_Virus","Tomato_Bacterial_spot",
                    "Tomato_Early_blight","Tomato_healthy","Tomato_Late_blight","Tomato_Leaf_Mold","Tomato_Septoria_leaf_spot"]

    @app.get("/ping")
    async def ping():
        return "Hello, I am alive"
    

    def read_file_as_image(data) -> np.ndarray:
        image=np.array(Image.open(BytesIO(data)))
        return image

    @app.post("/predict")
    async def predict(
        file: UploadFile = File(...)
        ):
        image = read_file_as_image(await file.read())
        img_batch = np.expand_dims(image,0)

        prediction = Model.predict(img_batch)

        predicted_class = CLASS_NAMES[np.argmax(prediction[0])]
        confidence=np.max(prediction[0])
        
        return{
            'class' : predicted_class,
            'confidence': float(confidence)
        }
    
    if __name__ == "__main__":
        uvicorn.run(app, host='localhost', port=8000)

loop = asyncio.get_event_loop()
loop.run_until_complete(my_coroutine())

INFO:     Started server process [1715]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
2023-10-12 19:59:10.758325: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


1/1 [==============================] - 3s 3s/step
INFO:     127.0.0.1:33274 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1715]
